In [ ]:
import numpy as np
np.random.seed(2017)

import os
import time
#from resnet50_ import ResNet50
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten

from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
from numpy import genfromtxt

trainLabelscsv = genfromtxt('train_sample.csv',dtype=str, delimiter=',')



PATH='DR_train_model/train'
files=os.listdir(PATH)
img_data_list=[]
labels=[]


#files.sort(key=int)

y=trainLabelscsv.shape[0]
for i in files:
     #print(i)
     try:
       img_path = PATH+ '/'+ i 
       img = image.load_img(img_path, target_size=(224, 224))
       x = image.img_to_array(img)
       x = np.expand_dims(x, axis=0)
       x = preprocess_input(x)
       #print('Input image shape:', x.shape)
       img_data_list.append(x)
       for j in range(0,y):
           if i==trainLabelscsv[j][0]+'.jpeg':
                  #print(trainLabelscsv[j][1])
                  labels.append(trainLabelscsv[j][1])
     except:   
          pass   
             
                  

img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print ("i/p shape",img_data.shape)
#img_data=img_data.reshape(img_data.shape[1:])
print ("i/p shape",img_data.shape)
img_data=np.rollaxis(img_data,1,0)

print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)
num_of_samples = img_data.shape[0]


                  
                  
#labels
num_classes=5
Y = np_utils.to_categorical(labels, num_classes)
print("i/p shape",Y.shape)

x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

###########################################################################################################################


# Fine tune the resnet 50
#image_input = Input(shape=(224, 224, 3))
model = ResNet50(weights='imagenet',include_top=False)
#model.summary()
last_layer = model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(last_layer)
# add fully-connected & dropout layers
x = Dense(512, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu',name='fc-2')(x)
x = Dropout(0.5)(x)
# a softmax layer for 4 classes
out = Dense(num_classes, activation='softmax',name='output_layer')(x)

# this is the model we will train
custom_resnet_model2 = Model(inputs=model.input, outputs=out)

custom_resnet_model2.summary()

for layer in custom_resnet_model2.layers[:-6]:
	layer.trainable = False

custom_resnet_model2.layers[-1].trainable

custom_resnet_model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
hist = custom_resnet_model2.fit(X_train, y_train, batch_size=32, epochs=7, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_resnet_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

custom_resnet_model2.save('healthcare_resnet50_1.h5')

Using TensorFlow backend.


i/p shape (1050, 1, 224, 224, 3)
i/p shape (1050, 1, 224, 224, 3)
(1, 1050, 224, 224, 3)
(1050, 224, 224, 3)
i/p shape (1050, 5)


C:\Users\DELL-PC\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

Train on 840 samples, validate on 210 samples
Epoch 1/7
840/840 [==============================] - 1853s 2s/step - loss: 1.3055 - accuracy: 0.6500 - val_loss: 0.9920 - val_accuracy: 0.7286
Epoch 2/7
 96/840 [==>...........................] - ETA: 35:59 - loss: 1.0502 - accuracy: 0.7083